## **Impianti PV e impianti eolici** 

L'obiettivo di questo studio è quello di rilevare come un impianto fotovoltaico o eolico incida sull'efficientamento energetico, inoltre, quanto un malfunzionamento possa incidere su quest'ultimo. 
Esamineremo in particolare i valori di un anno sulla generazione di energia residenziale e sui dati metereologici

In [ ]:
#caricamento e analisi dei dataset
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns 
import datetime
import sklearn
from matplotlib import pyplot
from plotly.offline import plot, iplot, init_notebook_mode
init_notebook_mode(connected=True)
from sklearn.metrics import r2_score,mean_squared_error,mean_absolute_error
from scipy.optimize import curve_fit
from pandas.tseries.offsets import DateOffset

! pip install pyspark
import pyspark
from pyspark.sql import SparkSession
from pyspark.conf import SparkConf
from pyspark import SparkContext
from pyspark.sql.functions import substring
from pyspark.sql.types import IntegerType
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import GBTRegressor

import warnings
warnings.filterwarnings('ignore')
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
#apertura e lettura dei vari dataset presi in considerazione 
solar = pd.read_csv(r'../input/residential-solar-vs-weather-nc-usa/Weather_and_energy_Final_2020_2021.csv', header=0)
panels = pd.read_csv(r'../input/residential-solar-vs-weather-nc-usa/Panel_Monthly_totals_2020_2021.csv', header=0)

p1gd = pd.read_csv("../input/solar-power-generation-data/Plant_1_Generation_Data.csv")
p2gd = pd.read_csv("../input/solar-power-generation-data/Plant_2_Generation_Data.csv")
#lettura del tempo metereologico
p1wd = pd.read_csv('../input/solar-power-generation-data/Plant_1_Weather_Sensor_Data.csv')
p1wd.drop('PLANT_ID',1,inplace=True)
p2wd = pd.read_csv('../input/solar-power-generation-data/Plant_2_Weather_Sensor_Data.csv')
p2wd.drop('PLANT_ID',1,inplace=True)

p1gd['DATE_TIME']= pd.to_datetime(p1gd['DATE_TIME'],format='%d-%m-%Y %H:%M')
p1wd['DATE_TIME']= pd.to_datetime(p1wd['DATE_TIME'],format='%Y-%m-%d %H:%M:%S')

p2gd['DATE_TIME']= pd.to_datetime(p2gd['DATE_TIME'],format='%Y-%m-%d %H:%M')
p2wd['DATE_TIME']= pd.to_datetime(p2wd['DATE_TIME'],format='%Y-%m-%d %H:%M:%S')

In [ ]:
print(solar.head())
print('_______________________')
print(panels.head())


Attraverso la distribuzione dei dati possiamo osservare la generazione di energia e le varie condizioni metereologiche. Si hanno bassi livelli di energia in prossimità di condizioni meteo nuuvolose. Le condizioni di scarico di energia rimangono distrete.

# **Impianto 1 e Impianto 2 - I confronti**
Analizziamo i due impianti fotovoltaici, dove SOURCE_KEY è la singola unità di condizionamento.
In questo caso i dati generati sono registrati con un intevallo metereologico di 15 minuti.

Possiamo subito notare che gli inverter dell'impianto 1 appaiono ampiamente stabili nella potenza AC generata. 
Gli inverter dell'impianto 2, tuttavia, fluttuano selvaggiamente. Ciò potrebbe essere dovuto a diversi fattori, ma è probabilme che la manutenzione possa essere un problema di quest'ultimo impianto. Un rapido sguardo alla deviazione standard dei dati in queste cifre quantifica meglio la misura in cui i dati fluttuano.

In [ ]:

pg1_inverters=p1gd['SOURCE_KEY'].nunique()
pg2_inverters=p2gd['SOURCE_KEY'].nunique()
print(f"L'impianto 1 ha  {pg1_inverters} inverter.")
print(f"L'impianto 2 ha {pg2_inverters} inverter.")

print(f"")
print(f"Impianto 1 Media corrente alternata (AC) per ogni inverter")
ax=sns.barplot(data=p1gd,x='SOURCE_KEY',y='AC_POWER',color='green',ci=None)


plt.xticks([])
ax.set(ylabel='Media corrente alternata AC(kW)')
plt.show()
print(f"")
print(f"Impianto 2 Media corrente alternata (AC) per ogni inverter")
ax=sns.barplot(data=p2gd,x='SOURCE_KEY',y='AC_POWER',color='green',ci=None)
plt.xticks([])
ax.set(ylabel='Media corrente alternata AC(kW)')
plt.show()

In [ ]:
s1 = p1gd.groupby('SOURCE_KEY').mean()['AC_POWER'].std()
print(f"Gli inverter dell'impianto 1 sono relativamente "
      f'stabili con solo una deviazione standard di {s1:.2f} kW nella media')

s2 = p2gd.groupby('SOURCE_KEY').mean()['AC_POWER'].std()
print(f"Gli inverter dell'impianto 2 sono relativamente "
      f'instabili con solo una deviazione standard di {s2:.2f} kW nella media')

In [ ]:
p1gd


In [ ]:
p2gd

Analizziamo i due **ambienti** dei due impianti, dove SOURCE_KEY è l'unità di monitoraggio metereologico in posizione ottimale dei singoli impianti.

In [ ]:
p1wd

In [ ]:
p2wd

In [ ]:
df_gen1=p1gd.groupby('DATE_TIME').sum().reset_index()
df_gen1['time']=df_gen1['DATE_TIME'].dt.time

fig,ax = plt.subplots(ncols=2,nrows=1,dpi=100,figsize=(20,5))
# daily yield plot
df_gen1.plot(x='DATE_TIME',y='DAILY_YIELD',color='navy',ax=ax[0])
# AC & DC power plot
df_gen1.set_index('time').drop('DATE_TIME',1)[['AC_POWER','DC_POWER']].plot(style='o',ax=ax[1])

ax[0].set_title('Rendimento giornaliero',)
ax[1].set_title('Energia AC/DC durante le ore diurne')
ax[0].set_ylabel('kW',color='navy',fontsize=17)
plt.show()

In [ ]:
df_gen3=df_gen1.copy();
df_gen3['time']=df_gen1['DATE_TIME'].dt.time
df_gen3['DC_POWER(AGG)']=df_gen3['DC_POWER']/10

df_gen3.max()

In [ ]:
fig,ax = plt.subplots(ncols=2,nrows=1,dpi=100,figsize=(20,5))
# daily yield plot
df_gen3.plot(x='DATE_TIME',y='DAILY_YIELD',color='navy',ax=ax[0])
# AC & DC power plot
df_gen3.set_index('time').drop('DATE_TIME',1)[['AC_POWER','DC_POWER(AGG)']].plot(style='o',ax=ax[1])

ax[0].set_title('Rendimento giornaliero',)
ax[1].set_title('Energia AC/DC durante le ore diurne')
ax[0].set_ylabel('kW',color='navy',fontsize=17)
plt.show()

In [ ]:
df_gen2=p2gd.groupby('DATE_TIME').sum().reset_index()
df_gen2['time']=df_gen2['DATE_TIME'].dt.time

fig,ax = plt.subplots(ncols=2,nrows=1,dpi=100,figsize=(20,5))
# daily yield plot
df_gen2.plot(x='DATE_TIME',y='DAILY_YIELD',color='navy',ax=ax[0])
# AC & DC power plot
df_gen2.set_index('time').drop('DATE_TIME',1)[['AC_POWER','DC_POWER']].plot(style='o',ax=ax[1])

ax[0].set_title('Rendimento giornaliero',)
ax[1].set_title('Energia AC/DC durante le ore diurne')
ax[0].set_ylabel('kW',color='navy',fontsize=17)
plt.show()

In [ ]:
p1gd.describe()

Sembra che l'alimentazione AC e l'alimentazione DC non siano nella stessa unità di potenza. Perché in genere l'efficienza dell'inverter (AC/DC) è superiore al 95%, il che non è questo il caso.



In [ ]:
p2gd.describe()

Nell' Impianto 2 la generazione minima di corrente continua e alternata (AC/DC) è 0kW in assenza di luce solare/irradiazione.
La massima generazione di potenza DC è 1420,93 kW associata a 1 SOURCE_KEY/Inverter.
Il rendimento giornaliero massimo per inverter è 9873 kWHr.

In [ ]:
losses=p1gd.copy()
losses['day']=losses['DATE_TIME'].dt.date
losses=losses.groupby('day').sum()
losses['losses']=losses['AC_POWER']/losses['DC_POWER']*100

losses['losses'].plot(style='o--',figsize=(17,5),label='Potenza effettiva')

plt.title('% di potenza DC convertita in corrente alternata AC',size=17)
plt.ylabel('Potenza DC  convertita (%)',fontsize=14,color='red')
plt.axhline(losses['losses'].mean(),linestyle='--',color='gray',label='Media')
plt.legend()
plt.show()

In [ ]:
losses.mean()

In [ ]:
losses=p2gd.copy()
losses['day']=losses['DATE_TIME'].dt.date
losses=losses.groupby('day').sum()
losses['losses']=losses['AC_POWER']/losses['DC_POWER']*100

losses['losses'].plot(style='o--',figsize=(17,5),label='Potenza effettiva')

plt.title('% di potenza DC convertita in corrente alternata AC',size=17)
plt.ylabel('Potenza DC  convertita (%)',fontsize=14,color='red')
plt.axhline(losses['losses'].mean(),linestyle='--',color='gray',label='Media')
plt.legend()
plt.show()

In [ ]:
losses

In [ ]:
dc_gen=p1gd.copy()
dc_gen['time']=dc_gen['DATE_TIME'].dt.time
dc_gen=dc_gen.groupby(['time','SOURCE_KEY'])['DC_POWER'].mean().unstack()

cmap = sns.color_palette("Set2", n_colors=12)

fig,ax=plt.subplots(ncols=2,nrows=1,dpi=100,figsize=(20,6))
dc_gen.iloc[:,0:11].plot(ax=ax[0],color=cmap)
dc_gen.iloc[:,11:22].plot(ax=ax[1],color=cmap)

ax[0].set_title('Primi 11 inverter')
ax[0].set_ylabel('DC POWER ( kW )',fontsize=17,color='navy')
ax[1].set_title('Ultimi 11 inverter')
plt.show()

Possiamo notare chiaramente che **1BY6WEcLGh8j5v7** e **bvBOhCH3iADSZry** hanno prestazioni inferiori ad altri inverter.
Forse questi inverter richiedono manutenzione o devono essere sostituiti. 

In [ ]:
# Import all available data 
df_gen1 = pd.read_csv("../input/solar-power-generation-data/Plant_1_Generation_Data.csv")
df_gen2 = pd.read_csv("../input/solar-power-generation-data/Plant_2_Generation_Data.csv")
df_weather1 = pd.read_csv("../input/solar-power-generation-data/Plant_1_Weather_Sensor_Data.csv")
df_weather2 = pd.read_csv("../input/solar-power-generation-data/Plant_2_Weather_Sensor_Data.csv")
# adjust datetime format
df_gen1['DATE_TIME'] = pd.to_datetime(df_gen1['DATE_TIME'],format = '%d-%m-%Y %H:%M')
df_weather1['DATE_TIME'] = pd.to_datetime(df_weather1['DATE_TIME'],format = '%Y-%m-%d %H:%M:%S')

# drop unnecessary columns and merge both dataframes along DATE_TIME
df_plant1 = pd.merge(df_gen1.drop(columns = ['PLANT_ID']), df_weather1.drop(columns = ['PLANT_ID', 'SOURCE_KEY']), on='DATE_TIME')
# add inverter number column to dataframe
sensorkeys = df_plant1.SOURCE_KEY.unique().tolist() # unique sensor keys
sensornumbers = list(range(1,len(sensorkeys)+1)) # sensor number
dict_sensor = dict(zip(sensorkeys, sensornumbers)) # dictionary of sensor numbers and corresponding keys

# add column
df_plant1['SENSOR_NUM'] = 0
for i in range(df_gen1.shape[0]):
    df_plant1['SENSOR_NUM'][i] = dict_sensor[df_gen1["SOURCE_KEY"][i]]

# add Sensor Number as string
df_plant1["SENSOR_NAME"] = df_plant1["SENSOR_NUM"].apply(str) # add string column of sensor name
# adding separate time and date columns
df_plant1["DATE"] = pd.to_datetime(df_plant1["DATE_TIME"]).dt.date # add new column with date
df_plant1["TIME"] = pd.to_datetime(df_plant1["DATE_TIME"]).dt.time # add new column with time

# add hours and minutes for ml models
df_plant1['HOURS'] = pd.to_datetime(df_plant1['TIME'],format='%H:%M:%S').dt.hour
df_plant1['MINUTES'] = pd.to_datetime(df_plant1['TIME'],format='%H:%M:%S').dt.minute
df_plant1['MINUTES_PASS'] = df_plant1['MINUTES'] + df_plant1['HOURS']*60

# add date as string column
df_plant1["DATE_STR"] = df_plant1["DATE"].astype(str) # add column with date as string
df_plant1.head()

Sembra che in alcuni giorni ci siano stati problemi con l'impianto:
tra il 19-05-2020 e il 21-05-2020 possiamo vedere un periodo con degli sbalzi nelle curve dovuti a malfunzionamenti, nella generazione di corrente continua. Quindi è confermato un guasto negli inverter.

In [ ]:
fig = px.scatter(df_plant1, x="TIME", y="DC_POWER", title="DC Power: Daily Distribution", color = "DATE_STR")
fig.update_traces(marker=dict(size=5, opacity=0.7), selector=dict(mode='markers'))
fig.show()

In [ ]:
fig = px.scatter(df_plant1, x="TIME", y="IRRADIATION", title="Irradiation: Daily Distribution", color = "DATE_STR")
fig.update_traces(marker=dict(size=5, opacity=0.7), selector=dict(mode='markers'))
fig.show()

In [ ]:

temp_gen1=p1gd.copy()

temp_gen1['time']=temp_gen1['DATE_TIME'].dt.time
temp_gen1['day']=temp_gen1['DATE_TIME'].dt.date

temp_sens1=p1wd.copy()

temp_sens1['time']=temp_sens1['DATE_TIME'].dt.time
temp_sens1['day']=temp_sens1['DATE_TIME'].dt.date


p1gd_time_grp= temp_gen1.groupby(['time']).sum()
p1gd_time_grp




In [ ]:
temp_gen2=p2gd.copy()
temp_gen2['time']=temp_gen2['DATE_TIME'].dt.time
temp_gen2['day']=temp_gen2['DATE_TIME'].dt.date
temp_sens2=p1wd.copy()
temp_sens2['time']=temp_sens2['DATE_TIME'].dt.time
temp_sens2['day']=temp_sens2['DATE_TIME'].dt.date
p2gd_time_grp = temp_gen2.groupby(['time']).sum()
p2gd_time_grp

In [ ]:
p1gd_time_grp['DAILY_YIELD'].plot(figsize=(20,5))
plt.title("Resa totale di 34 giorni dell'impianto 1 con l'ora indicativa")
plt.ylabel('Rendimento in kWH')



### Correlazione tra le variabili 

Innanzitutto, combiniamo le variabili metereologiche alle variabili dei pannelli fotovoltaici

In [ ]:
p1gd_DT = p1gd.groupby(['DATE_TIME'],as_index=False).sum()
p1gd_DT_Select = p1gd_DT[['DATE_TIME','DC_POWER','AC_POWER','DAILY_YIELD']]
p1gd_DT_Select
p1wd_drp = p1wd.drop([ 'SOURCE_KEY'], axis=1)
merged_data = pd.merge(p1gd_DT_Select, p1wd_drp, how='inner', on='DATE_TIME')
merged_data['time']=merged_data['DATE_TIME'].dt.time
merged_data.iloc[25:35]

In [ ]:
merged_data_Irr = merged_data.copy()
merged_data_Irr_t = merged_data_Irr.groupby(['time']).mean()

fig,ax=plt.subplots(ncols=2,nrows=2,dpi=200,figsize=(15,5))
merged_data_Irr_t['IRRADIATION'].plot(ax=ax[0,0])
merged_data_Irr_t['AMBIENT_TEMPERATURE'].plot(ax=ax[0,1])
merged_data_Irr_t['MODULE_TEMPERATURE'].plot(ax=ax[1,0])
merged_data_Irr_t['DC_POWER'].plot(ax=ax[1,1])

ax[0,0].set_ylabel('IRRADIATION')
ax[0,1].set_ylabel('AMBIENT TEMPERATURE')
ax[1,0].set_ylabel('MODULE TEMPERATURE')
ax[1,1].set_ylabel('DC POWER')

Possiamo osservare dai grafici come mediamente Irradiazione, Temperatura Ambiente, Temperatura Modulo e Potenza DC prima aumentano e poi diminuiscono dall'alba al tramonto.
Generalmente la temperatura del modulo aumenta con l'aumento della temperatura e dell'irraggiamento.


In [ ]:
merged_data_num = merged_data[['DC_POWER','AC_POWER','DAILY_YIELD','AMBIENT_TEMPERATURE','MODULE_TEMPERATURE','IRRADIATION']]
corr = merged_data_num.corr()
fig_dims = (2, 2) 
sns.heatmap(round(corr,2), annot=True, mask=(np.triu(corr,+1)))

La generazione di corrente continua è correlata in modo altamente positivo con l'irradiazione, la temperatura del modulo e la temperatura ambiente.

Tuttavia, non si dovrebbe concludere qui che la generazione di energia DC aumenta con l'aumento della temperatura del modulo poiché sappiamo che l'efficienza del modulo solare diminuisce con l'aumento della temperatura. 

# Influenza degli agenti atmosferici sui pannelli

In [ ]:
plt.figure(figsize=(10,8))
solarcorr = solar.corr()
sns.heatmap(solarcorr, cmap='YlGnBu', annot=True)
plt.show()

Dalla heatmap possiamo notare che la copertura nuvolosa (Cloud Cover) è il valore più correlato con la scarica di energia (Energy Dischaged).

In [ ]:
#Genera una linea di regressione con un adattamento per escludere i valori anomali
import seaborn as sns; sns.set_theme(color_codes=True)
ax = sns.regplot(x='Cloud Cover', y='Energy Discharged (Wh)', data=solar, robust=True)

Possiamo chiaramente notare dunque, che la copertura del cielo può enormemente influenzare l'energia scaricata dai pannelli.

## Predizioni

In [ ]:
from fbprophet import Prophet
pred_gen=p1gd.copy()
pred_gen=pred_gen.groupby('DATE_TIME')['DAILY_YIELD'].sum().reset_index()
pred_gen.rename(columns={'DATE_TIME':'ds','DAILY_YIELD':'y'},inplace=True)
pred_gen.plot(x='ds',y='y',figsize=(20,10))
plt.title('DAILY_YIELD',size=17)
plt.show()

# Prediction LSTM

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas
import math
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error


In [ ]:

pred_g1=p1gd.copy()
pred_g1=pred_g1.groupby('DATE_TIME').sum()
pred_g1=pred_g1['DAILY_YIELD'].reset_index()
pred_g1.set_index('DATE_TIME',inplace=True)
pred_g1

train=pred_g1[:3158]
test=pred_g1[-1042:]
#67% e 33%
plt.figure(figsize=(20,10))
plt.plot(train,label='Addestramento')
plt.plot(test,label='Test', color='green')
plt.title('Giorni totali di rendimento giornaliero',fontsize=17)
plt.legend()
plt.show()

In [ ]:
def create_dataset(dataset, look_back=1):
	dataX, dataY = [], []
	for i in range(len(dataset)-look_back-1):
		a = dataset[i:(i+look_back), 0]
		dataX.append(a)
		dataY.append(dataset[i + look_back, 0])
	return np.array(dataX), np.array(dataY)

In [ ]:
# split into train and test sets
dataframe= pred_g1["DAILY_YIELD"]
dataset = dataframe.values
dataset = dataset.astype('float32')
dataset= dataset.reshape(-1, 1)

# normalize the dataset
scaler = MinMaxScaler(feature_range=(0, 1))
dataset = scaler.fit_transform(dataset)

# split into train and test sets
train_size = int(len(dataset) * 0.67)
test_size = len(dataset) - train_size

train= dataset[0:train_size]
test = dataset[train_size:len(dataset)]

look_back = 1
trainX, trainY = create_dataset(train, look_back)
testX, testY = create_dataset(test, look_back)

# reshape input to be [samples, time steps, features]
trainX = np.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
testX = np.reshape(testX, (testX.shape[0], 1, testX.shape[1]))

In [ ]:
# create and fit the LSTM network
model = Sequential()
model.add(LSTM(4, input_shape=(1, look_back)))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(trainX, trainY, epochs=3158, batch_size=1, verbose=2)

In [ ]:
# make predictions
trainPredict = model.predict(trainX)
testPredict = model.predict(testX)
# invert predictions
trainPredict = scaler.inverse_transform(trainPredict)
trainY = scaler.inverse_transform([trainY])
testPredict = scaler.inverse_transform(testPredict)
testY = scaler.inverse_transform([testY])


# calculate root mean squared error
trainScore = math.sqrt(mean_squared_error(trainY[0], trainPredict[:,0]))
print('Train Score: %.2f RMSE' % (trainScore))
testScore = math.sqrt(mean_squared_error(testY[0], testPredict[:,0]))
print('Test Score: %.2f RMSE' % (testScore))
print('-'*15)
print('MAE Train Score: %f' % (mean_absolute_error(trainY[0], trainPredict[:,0])))
print('MAE Test Score: %f' % (mean_absolute_error(testY[0], testPredict[:,0])))
print('-'*15)
print('Prophet Train R2 Score: %f' % (r2_score(trainY[0], trainPredict[:,0])))
print('Prophet Test R2 Score: %f' % (r2_score(testY[0], testPredict[:,0])))


In [ ]:
# shift train predictions for plotting
trainPredictPlot = np.empty_like(dataset)
trainPredictPlot[:, :] = np.nan
trainPredictPlot[look_back:len(trainPredict)+look_back, :] = trainPredict
# shift test predictions for plotting
testPredictPlot = np.empty_like(dataset)
testPredictPlot[:, :] = np.nan
testPredictPlot[len(trainPredict)+(look_back*2)+1:len(dataset)-1, :] = testPredict
# plot baseline and predictions
fig,ax= plt.subplots(ncols=1,nrows=1,dpi=100,figsize=(30,10))
plt.plot(scaler.inverse_transform(dataset))
plt.plot(trainPredictPlot)
plt.plot(testPredictPlot)
plt.show()

# Prediction in Prophet

In [ ]:
p = Prophet()
p.fit(pred_gen)

future = [pred_gen['ds'].iloc[-1:] + DateOffset(minutes=x) for x in range(0,2910,15)]
time1=pd.DataFrame(future).reset_index().drop('index',1)
time1.rename(columns={3157:'ds'},inplace=True)

timeline=pd.DataFrame(pred_gen['ds'])
fut=timeline.append(time1,ignore_index=True)

forecast = p.predict(fut)

In [ ]:
p.plot(forecast,figsize=(30,10))
plt.legend(labels=['Dati reali','Prediction'])
plt.title('Predizioni')
plt.show()

In [ ]:
fig1 = p.plot_components(forecast)

In [ ]:
from sklearn.metrics import r2_score,mean_squared_error,mean_absolute_error
test2=pd.DataFrame(pred_gen)
test2.rename(columns={'DATE_TIME':'ds'},inplace=True)
test_prophet=p.predict(test2)

In [ ]:
pred_gen2=p1gd.copy()
pred_gen2=pred_gen2.groupby('DATE_TIME')['DAILY_YIELD'].sum().reset_index()

In [ ]:
print('Prophet R2 Score: %f' % (r2_score(pred_gen2['DAILY_YIELD'],test_prophet['yhat'])))
print('-'*15)
print('Prophet MAE Score: %f' % (mean_absolute_error(pred_gen2['DAILY_YIELD'],test_prophet['yhat'])))
print('-'*15)
print('Prophet RMSE Score: %f' % (mean_squared_error(pred_gen2['DAILY_YIELD'],test_prophet['yhat'],squared=False)))

In [ ]:
# Funzione per controllare se l'ora è durante il funzionamento diurno
def time_in_range(start, end, x):
    """Return true if x is in the range [start, end]"""
    if start <= end:
        return start <= x <= end
    else:
        return start <= x or x <= end
    
start=datetime.time(6,30,0) # alba
end=datetime.time(17,30,0) # tramonto

# Crea una nuova colonna per verificare il corretto funzionamento
# Restituisce "Normale" se il funzionamento è normale e "Guasto" se il funzionamento è difettoso
df_plant1["STATUS"] = 0
for index in df_plant1.index:
    if  time_in_range(start, end, df_plant1["TIME"][index]) and df_plant1["DC_POWER"][index] == 0:
        df_plant1["STATUS"][index] = "Guasto"
    else:
        df_plant1["STATUS"][index] = "Normale"

## 

In [ ]:
df_plant1

In [ ]:
fig = px.scatter(df_plant1, x="IRRADIATION", y="DC_POWER", title="Identificazione dei guasti", color="STATUS", labels={"DC_POWER":"Potenza DC (kW)", "IRRADIATION":"Irradiazione"})
fig.update_traces(marker=dict(size=3, opacity=0.7), selector=dict(mode='marker'), color='navy')
fig.show()


In [ ]:
df_plant1[df_plant1["STATUS"]== "Guasto"]["DATE"].value_counts()

In [ ]:
df_plant1.STATUS.value_counts()

In [ ]:
fig=px.bar(df_plant1[df_plant1["STATUS"]== "Guasto"]["SOURCE_KEY"].value_counts(), title="Guasti inverter", labels={"value":"Guasti", "index":"Inverter", "SENSOR_NAME":"Inverter"})
fig.update(layout_showlegend=False)

In [ ]:
print("La maggior parte dei guasti è stata registrata su {} e {}." .format(df_plant1[df_plant1["STATUS"]== "Guasto"]["DATE"].value_counts().index[0], df_plant1[df_plant1["STATUS"]== "Guasto"]["DATE"].value_counts().index[1]))
print("Gli inverter {} e {} hanno avuto il maggior numero di guasti." .format(df_plant1[df_plant1["STATUS"]== "Guasto"]["SOURCE_KEY"].value_counts().index[0],df_plant1[df_plant1["STATUS"]== "Guasto"]["SOURCE_KEY"].value_counts().index[1]))

In [ ]:
from sklearn.linear_model import LinearRegression

# Modello
reg = LinearRegression()

# training data
train_dates = ["2020-05-16", "2020-05-17","2020-05-18" ,"2020-05-19", "2020-05-20", "2020-05-21"]
df_train = df_plant1[df_plant1["DATE_STR"].isin(train_dates)]

#previsione e adattamento
reg.fit(df_train[["IRRADIATION"]], df_train.DC_POWER)
prediction = reg.predict(df_plant1[["IRRADIATION"]])

# salva previsione, residuo e residuo assoluto
df_train["Prediction"] = reg.predict(df_train[["IRRADIATION"]])
df_train["Residual"] = df_train["Prediction"] - df_train["DC_POWER"]
df_plant1["Prediction"] = reg.predict(df_plant1[["IRRADIATION"]])
df_plant1["Residual"] = df_plant1["Prediction"] - df_plant1["DC_POWER"]
df_plant1["Residual_abs"] = df_plant1["Residual"].abs()

In [ ]:
def func(X, a, b, c, d):
    '''Funzione non lineare per prevedere l'uscita di potenza CC dall'irradiazione e dalla temperatura.'''
    x,y = X
    x=x*1000
    y=y*1000
    return a*x*(1-b*(y+x/800*(c-20)-25)-d*np.log(x+1e-10))

# funzione adattamento
p0 = [1.,0.,-1.e4,-1.e-1] # avvio valori
popt, pcov = curve_fit(func, (df_train.IRRADIATION, df_train.MODULE_TEMPERATURE), df_train.DC_POWER, p0, maxfev=5000)
sigma_abcd = np.sqrt(np.diagonal(pcov))

# predizioni e salvataggio
df_train["Prediction_NL"] = func((df_train.IRRADIATION, df_train.MODULE_TEMPERATURE), *popt)
df_train["Residual_NL"] = df_train["Prediction_NL"] - df_train["DC_POWER"]

df_plant1["Prediction_NL"] = func((df_plant1.IRRADIATION, df_plant1.MODULE_TEMPERATURE), *popt)
df_plant1["Residual_NL"] = df_plant1["Prediction_NL"] - df_plant1["DC_POWER"]

In [ ]:
plt.figure(figsize=(20,7))
plt.scatter(df_plant1.IRRADIATION, df_plant1.DC_POWER, label="Misurazioni")
plt.scatter(df_plant1.IRRADIATION, df_plant1.Prediction_NL, color="r", label="Predizioni NL")
plt.legend()
plt.xlabel("Irradiazione (kW/m²)")
plt.ylabel("Potenza DC(kW)")
plt.title("Modello Predizione NL")
plt.legend();

# Analisi della potenza di una turbina eolica a livello domestico (Microeolico)

In [ ]:
data_T_start=pd.read_csv("../input/wind-turbine-scada-dataset/T1.csv")
data_T_start

In [ ]:
data1_T=data_T_start.copy()
data_T_start.rename(columns={'LV ActivePower (kW)':'Potenza attiva (kW)',"Wind Speed (m/s)":"Velocità vento(m/s)","Theoretical_Power_Curve (KWh)":"Curva Potenza Teorica (KWh)","Wind Direction (°)":"Direzione Vento"},
                inplace=True)
data1_T.rename(columns={'LV ActivePower (kW)':'Potenza attiva (kW)',"Wind Speed (m/s)":"Velocità vento(m/s)","Theoretical_Power_Curve (KWh)":"Curva Potenza Teorica (KWh)","Wind Direction (°)":"Direzione Vento"},
                inplace=True)
data1_T.rename(columns={'Date/Time':'Tempo'},inplace=True)

In [ ]:
#funzione per ricerca del mese
def find_month(x):
    if " 01 " in x:
        return "Gen"
    elif " 02 " in x:
        return "Feb"
    elif " 03 " in x:
        return "Mar"    
    elif " 04 " in x:
        return "Apr"    
    elif " 05 " in x:
        return "Mag"    
    elif " 06 " in x:
        return "Giu"    
    elif " 07 " in x:
        return "Lug"    
    elif " 08 " in x:
        return "Ago"    
    elif " 09 " in x:
        return "Set"    
    elif " 10 " in x:
        return "Ott"    
    elif " 11 " in x:
        return "Nov"    
    else:
        return "Dic"    
    #aggiunge mese al dataset
data1_T['Mese']=data1_T.Tempo.apply(find_month)
data1_T.head()

In [ ]:
#funzione per riscrivere la velocità del vento per intervalli di 0,5.
def mean_speed(x):
    list=[]
    i=0.25
    while i<=25.5:
        list.append(i)
        i+=0.5
    for i in list:
        if x < i:
            x=i-0.25
            return x

In [ ]:
data1_T["Media Velocità vento"]=data1_T["Velocità vento(m/s)"].apply(mean_speed)
data1_T.head()

In [ ]:
#funzione per riscrivere la direzione del vento per 30 intervalli.
def mean_direction(x):
    list=[]
    i=15
    while i<=375:
        list.append(i)
        i+=30
        
    for i in list:
        if x < i:
            x=i-15
            if x==360:
                return 0
            else:
                return x

In [ ]:
data1_T["Direzione media"]=data1_T["Direzione Vento"].apply(mean_direction)
data1_T.head()

In [ ]:
#funzione per la direzione del vento
def find_direction(x):
    if x==0:
        return "NORD"
    if x==30:
        return "NNE"
    if x==60:
        return "NEE" 
    if x==90:
        return "EST" 
    if x==120:
        return "SEE" 
    if x==150:
        return "SSE" 
    if x==180:
        return "SUD" 
    if x==210:
        return "SSW" 
    if x==240:
        return "SWW" 
    if x==270:
        return "OVEST" 
    if x==300:
        return "NWW" 
    if x==330:
        return "NNW"
  

In [ ]:
data1_T["Direzione"]=data1_T["Direzione media"].apply(find_direction)
data1_T.head()

In [ ]:
#Rimuoviamo i dati che indicano che la velocità del vento è inferiore a 3,5 e superiore a 25,5
#Lo facciamo poichè secondo la curva di potenza della turbina a livello domestico, la turbina lavora solo tra questi valori.
data2_T=data1_T[(data1_T["Velocità vento(m/s)"]>3.5) & (data1_T["Velocità vento(m/s)"]<=25.5)]

In [ ]:
#Numero di valori in cui la velocità del vento è maggiore di 3,5 e la potenza attiva è 0.
#Se la velocità del vento è maggiore di 3,5 e la potenza attiva è zero, significa che la turbina è fuori servizio.

len(data2_T["Potenza attiva (kW)"][(data2_T["Potenza attiva (kW)"]==0)&(data2_T["Velocità vento(m/s)"]>3.5)])


In [ ]:
data3_T=data2_T[((data2_T["Potenza attiva (kW)"]!=0)&(data2_T["Velocità vento(m/s)"]>3.5)) | (data2_T["Velocità vento(m/s)"]<=25.5)]

In [ ]:
from math import radians

plt.figure(figsize=(8,8))
ax = plt.subplot(111, polar=True)
# All'interno dei cerchi ci sono la velocità del vento e il colore e la dimensione dell'indicatore rappresenta la quantità di produzione di energia
sns.scatterplot(x=[radians(x) for x in data2_T['Direzione Vento']], 
                y=data2_T['Velocità vento(m/s)'],
                size=data2_T['Potenza attiva (kW)'],
                hue=data2_T['Potenza attiva (kW)'],
                alpha=0.7, legend=None)
# L'impostazione della parte superiore del diagramma polare rappresenta il Nord
ax.set_theta_zero_location('N')
# Imposto -1 per avviare la direzione del vento in senso orario
ax.set_theta_direction(-1)
ax.set_rlabel_position(110)
plt.title('Velocità del vento - Direzione del vento - Diagramma di produzione di energia \n\n\n')
plt.ylabel(None);

A questo punto il nostro obbiettivo è calcolare la perdita. La perdita è la differenza tra Nordex_Powercurve e la potenza attiva.

In [ ]:
data3_T["Curva Potenza Teorica (KWh)"][data3_T["Media Velocità vento"]==5.5].mean()

In [ ]:
data_T_clean=data3_T.sort_values("Tempo")
data_T_clean["Valore Perdita(kW)"]=data_T_clean["Curva Potenza Teorica (KWh)"]-data_T_clean["Potenza attiva (kW)"]
data_T_clean["Perdita(%)"]=data_T_clean["Valore Perdita(kW)"]/data_T_clean["Curva Potenza Teorica (KWh)"]*100
data_T_clean=data_T_clean.round({'Potenza attiva (kW)': 2, 'Velocità vento(m/s)': 2, 'Curva Potenza Teorica (KWh)': 2,
                                   'Direzione Vento': 2, 'Valore Perdita(kW)': 2, 'Perdita': 2})

data_T_clean.head()

In [ ]:
#creazione di dataframe di velocità di riepilogo da dati puliti.
DepGroupT_speed = data_T_clean.groupby("Media Velocità vento")
data_T_speed=DepGroupT_speed.mean()
#rimuoviamo le colonne non necessarie
data_T_speed.drop(columns={"Velocità vento(m/s)","Direzione Vento","Direzione media"},inplace=True)
#creiamo una colonna della velocità del vento dai valori dell'indice.
listTspeed_WS=data_T_speed.index.copy()
data_T_speed["Velocità vento(m/s)"]=listTspeed_WS

data_T_speed=data_T_speed[["Velocità vento(m/s)","Potenza attiva (kW)","Curva Potenza Teorica (KWh)","Valore Perdita(kW)","Perdita(%)"]]
data_T_speed["Index"]=list(range(1,len(data_T_speed.index)+1))
data_T_speed.set_index("Index",inplace=True)
data_T_speed=data_T_speed.round({"Velocità vento(m/s)": 1, 'Potenza attiva (kW)': 2, 'Curva Potenza Teorica (KWh)': 2, 'Valore Perdita(kW)': 2, 'Perdita(%)': 2})
#creando una colonna di conteggio che mostra il numero di velocità del vento da dati puliti.
data_T_speed["count"]=[len(data_T_clean["Media Velocità vento"][data_T_clean["Media Velocità vento"]==i]) 
                        for i in data_T_speed["Velocità vento(m/s)"]]
data_T_speed

In [ ]:
#creazione di dataframe di velocità di riepilogo da dati puliti.
DepGroupT_direction = data_T_clean.groupby("Direzione")
data_T_direction=DepGroupT_direction.mean()
#rimuoviamo le colonne non necessarie
data_T_direction.drop(columns={"Velocità vento(m/s)","Direzione Vento"},inplace=True)
#creo un index
listTdirection_Dir=data_T_direction.index.copy()
data_T_direction["Direzione"]=listTdirection_Dir

data_T_direction["Velocità vento(m/s)"]=data_T_direction["Media Velocità vento"]
data_T_direction.drop(columns={"Media Velocità vento"},inplace=True)
#cambiamo il posto delle colonne.
data_T_direction=data_T_direction[["Direzione","Direzione media","Potenza attiva (kW)","Curva Potenza Teorica (KWh)","Velocità vento(m/s)",
                                     "Valore Perdita(kW)","Perdita(%)"]]
#cambiamo l'indice dei numeri
data_T_direction["Index"]=list(range(1,len(data_T_direction.index)+1))
data_T_direction.set_index("Index",inplace=True)
#create a count column that shows the number of directions from clean data.
data_T_direction["count"]=[len(data_T_clean["Direzione"][data_T_clean["Direzione"]==i]) 
                        for i in data_T_direction["Direzione"]]
#arrotondiamo i valori a 2 cifre
data_T_direction=data_T_direction.round({'Velocità vento(m/s)': 1,'Potenza attiva (kW)': 2, 'Curva Potenza Teorica (KWh)': 2,
                                           'Valore Perdita(kW)': 2, 'Perdita(%)': 2})

data_T_direction=data_T_direction.sort_values("Direzione media")
data_T_direction.drop(columns={"Direzione media"},inplace=True)
data_T_direction

In [ ]:
#Disegno grafico delle potenze medie in base alla direzione del vento.
def bar_graph():
    turbine_no="T1"
    fig = plt.figure(figsize=(20,10))
    plt.bar(data_T_direction["Direzione"],data_T_direction["Curva Potenza Teorica (KWh)"],label="Curva Potenza Teorica (KWh)",align="edge",width=0.3)
    plt.bar(data_T_direction["Direzione"],data_T_direction["Potenza attiva (kW)"],label="Curva di potenza effettiva",align="edge",width=-0.3)
    plt.xlabel("Direzione Vento")
    plt.ylabel("Potenza (kW)")
    plt.title("Impianto eolico {} Valori medi di potenza rispetto alla direzione".format(turbine_no))
    plt.legend()
    plt.show()
bar_graph()

In [ ]:
#creazione di dataframe di velocità di riepilogo da dati puliti.
data_T_direction_total=data_T_direction.copy()
#rimuoviamo le colonne non necessarie
data_T_direction_total.drop(columns={"count","Potenza attiva (kW)","Curva Potenza Teorica (KWh)","Valore Perdita(kW)","Perdita(%)"},inplace=True)

data_T_direction_total["Generazione totale(MWh)"]=data_T_direction["Potenza attiva (kW)"]*data_T_direction["count"]/6000
data_T_direction_total["Generazione teorica totale(MWh)"]=data_T_direction["Curva Potenza Teorica (KWh)"]*data_T_direction["count"]/6000
data_T_direction_total["Perdita Totale(MWh)"]=data_T_direction_total["Generazione teorica totale(MWh)"]-data_T_direction_total["Generazione totale(MWh)"]
data_T_direction_total["Perdita(%)"]=data_T_direction_total["Perdita Totale(MWh)"]/data_T_direction_total["Generazione teorica totale(MWh)"]*100
#arrotondiamo i valori a 2 cifre
data_T_direction_total=data_T_direction_total.round({'Velocità vento(m/s)': 1,'Generazione totale(MWh)': 2, 'Generazione teorica totale(MWh)': 2,
                                           'Perdita Totale(MWh)': 2, 'Perdita(%)': 2})
#cambiamo il posto delle colonne.
data_T_direction_total=data_T_direction_total[["Direzione","Generazione totale(MWh)","Generazione teorica totale(MWh)","Velocità vento(m/s)",
                                     "Perdita Totale(MWh)","Perdita(%)"]]
data_T_direction_total

In [ ]:
def bar_graph():
    turbine_no="T1"
    fig = plt.figure(figsize=(20,10))
    plt.bar(data_T_direction_total["Direzione"],data_T_direction_total["Generazione teorica totale(MWh)"],label="Curva Potenza Teorica (KWh)",align="edge",width=0.3)
    plt.bar(data_T_direction_total["Direzione"],data_T_direction_total["Generazione totale(MWh)"],label="Curva di potenza effettiva",align="edge",width=-0.3)
    plt.xlabel("Direzione Vento")
    plt.ylabel("Generazione Energia (MWh)")
    plt.title("Impianto eolico {} Valori di generazione totale di energia rispetto alla direzione".format(turbine_no))
    plt.legend()
    plt.show()
bar_graph()

In [ ]:
#Disegno grafico della perdita totale in base alla direzione del vento.
def bar_graph():
    turbine_no="T1"
    fig = plt.figure(figsize=(20,10))
    plt.bar(data_T_direction_total["Direzione"],data_T_direction_total["Perdita Totale(MWh)"],
            label="Perdita totale(MWh)",align="center",width=0.5, color="lightgreen",picker=5)
    plt.xlabel("Direzione Vento")
    plt.ylabel("Totale perdita(MWh)")
    plt.title("Impianto eolico {} Valori di perdita totale rispetto alla direzione".format(turbine_no))
    plt.legend()
    plt.show()
bar_graph()

In [ ]:
#correlazione tra i valori
corr = data_T_start.corr()
plt.figure(figsize=(10, 8))
ax = sns.heatmap(corr, vmin = -1, vmax = 1, annot = True)
bottom, top = ax.get_ylim()
ax.set_ylim(bottom + 0.5, top - 0.5)
plt.show()
corr

## Prediction 

In [ ]:
# Configuraione Spark
spark = SparkSession.builder.master("local").appName("wind_turbine_project").getOrCreate()
sc = spark.sparkContext
# Lettura dataset come Spark DataFrame
Data1_start = spark.read.csv('../input/wind-turbine-scada-dataset/T1.csv', header=True, inferSchema=True)
Data1_start.cache()
Data1_start = Data1_start.toDF(*[c.lower() for c in Data1_start.columns])
# Estrazione mese ora e giorno dallo Spark

Data1_start = Data1_start.withColumn("month", substring("date/time", 4,2))
Data1_start = Data1_start.withColumn("hour", substring("date/time", 12,2))


# Conversione in intero 
Data1_start = Data1_start.withColumn('month', Data1_start.month.cast(IntegerType()))
Data1_start = Data1_start.withColumn('hour', Data1_start.hour.cast(IntegerType()))
# Campione casuale dai big data
sample_df = Data1_start.sample(withReplacement=False, fraction=0.1, seed=42).toPandas()

# lv activepower (kw) in label
Data1_start = Data1_start.withColumn('label', Data1_start['lv activepower (kw)'])
# variabili da utilizzare
variables = ['month', 'hour', 'wind speed (m/s)', 'wind direction (°)']
vectorAssembler = VectorAssembler(inputCols = variables, outputCol = 'features')
va_df = vectorAssembler.transform(Data1_start)

final_df = va_df.select('features', 'label')
splits = final_df.randomSplit([0.8, 0.2])
train_df = splits[0]
test_df = splits[1]

# creazione modello GBM
gbm = GBTRegressor(featuresCol='features', labelCol='label')

#Allenamento del modello
gbm_model = gbm.fit(train_df)

# Predizioni sul test_df
y_pred = gbm_model.transform(test_df)


In [ ]:
Data1_start.select('wind speed (m/s)', 'lv activepower (kw)')\
.filter(Data1_start['wind speed (m/s)'] >= 20)\
.filter(Data1_start['wind speed (m/s)'] <= 25 )\
.agg({'lv activepower (kw)':'mean'}).show()

In [ ]:
print('Train dataset: ', train_df.count())
print('Test dataset : ', test_df.count())

In [ ]:
test_df

In [ ]:
# Initial model success
from pyspark.ml.evaluation import RegressionEvaluator

evaluator = RegressionEvaluator(predictionCol='prediction', labelCol='label')

print('R2 SCORE : ', evaluator.evaluate(y_pred, {evaluator.metricName: 'r2'}))
print('MAE      : ', evaluator.evaluate(y_pred, {evaluator.metricName: 'mae'}))
print('RMSE     : ', evaluator.evaluate(y_pred, {evaluator.metricName: 'rmse'}))

In [ ]:
# Converto sample_df in Spark dataframe
eva_df = spark.createDataFrame(sample_df)
#  lv activepower (kw) in label
eva_df = eva_df.withColumn('label', eva_df['lv activepower (kw)'])
# Variabili da utilizzare
variables = ['month', 'hour', 'wind speed (m/s)', 'wind direction (°)']
vectorAssembler = VectorAssembler(inputCols = variables, outputCol = 'features')
vec_df = vectorAssembler.transform(eva_df)
# Combino features e label
vec_df = vec_df.select('features', 'label')
# Modello ML 
preds = gbm_model.transform(vec_df)
preds_df = preds.select('label','prediction').toPandas()

# Concateno i dataframes 
frames = [sample_df[['wind speed (m/s)', 'theoretical_power_curve (kwh)']], preds_df]
sample_data = pd.concat(frames, axis=1)

# Visual Potenza Reale, teorica e predizione 
plt.figure(figsize=(10,7))
sns.scatterplot(x='wind speed (m/s)', y='label',alpha=0.5, label= 'Potenza reale', data=sample_data)
sns.scatterplot(x='wind speed (m/s)', y='prediction', alpha=0.7, label='Predizione', marker='o', color='pink', data=sample_data)
sns.lineplot(x='wind speed (m/s)', y='theoretical_power_curve (kwh)', label='Potenza Teorica',color='purple', data=sample_data)
plt.title('Previsione della produzione di energia della turbina eolica')
plt.ylabel('Produzione di energia (kw)')
plt.xlabel('Velocità vento(m/s)')
plt.legend();

In [ ]:
MedOraria = Data1_start.groupby('hour').mean('lv activepower (kw)').sort('avg(lv activepower (kw))').toPandas()
sns.barplot(x='hour', y='avg(lv activepower (kw))', data=MedOraria)
plt.xlabel('Orario')
plt.ylabel('Media Potenza Attiva (Kw)')
plt.title('Produzione media oraria di energia');

In [ ]:
# Average power production by month
monthly = Data1_start.groupby('month').mean('lv activepower (kw)').sort('avg(lv activepower (kw))').toPandas()
sns.barplot(x='month', y='avg(lv activepower (kw))', data=monthly)
plt.xlabel('Mese')
plt.ylabel('Media Potenza Attiva (Kw)')
plt.title('Media Potenza Attiva mensile');

In [ ]:
sample_df['DATE_TIME']= pd.to_datetime(sample_df['date/time'],format='%d %m %Y %H:%M')
sample_df["DATE"] = pd.to_datetime(sample_df["DATE_TIME"]).dt.date # add new column with date

sample_df['Mese']=pd.to_datetime(sample_df["DATE_TIME"]).dt.month
sample_df['Giorno']=pd.to_datetime(sample_df["DATE_TIME"]).dt.day 
sample_df['hour']=pd.to_datetime(sample_df["DATE_TIME"]).dt.hour


FinalPred=sample_df.drop('month', axis = 1)
FinalPred=sample_df.drop('hour', axis = 1)
FinalPred


In [ ]:
# Converto sample_df in Spark dataframe
eva_df = spark.createDataFrame(FinalPred)
#  lv activepower (kw) in label
eva_df = eva_df.withColumn('label', eva_df['lv activepower (kw)'])
# Variabili da utilizzare
variables = ['Mese', 'Giorno', 'wind speed (m/s)', 'wind direction (°)']
vectorAssembler = VectorAssembler(inputCols = variables, outputCol = 'features')
vec_df = vectorAssembler.transform(eva_df)
# Combining features and label column
vec_df = vec_df.select('features', 'label')
# Modello ML 
preds = gbm_model.transform(vec_df)
preds_df = preds.select('label','prediction').toPandas()

# Concateno i  dataframes 
frames = [FinalPred[['Mese', 'Giorno','wind speed (m/s)', 'theoretical_power_curve (kwh)', "DATE"]], preds_df]
sample_data1 = pd.concat(frames, axis=1)
sample_data1= sample_data1.groupby(['DATE']).sum()
sample_data1

In [ ]:
y_actual= sample_data1["label"]
y_predicted= sample_data1["prediction"]
y_theorical= sample_data1["theoretical_power_curve (kwh)"]

# Visual Potenza Reale, teorica e predizione 
plt.figure(figsize=(30,10))
sns.lineplot(x='DATE', y=y_actual, label= 'Potenza reale',color='navy', data=sample_data1)
sns.lineplot(x='DATE', y=y_theorical, label='Potenza Teorica',color='red', data=sample_data1)
sns.lineplot(x='DATE', y=y_predicted, label='Predizione', color='Green', data=sample_data1)
plt.title('Previsione della produzione di energia della turbina eolica')
plt.ylabel('Produzione di energia (kw)')
plt.xlabel('Giorno')
plt.legend();

In [ ]:
print(' R2 Score: %f' % (r2_score(sample_data1["label"],sample_data1["prediction"])))
print('-'*15)
print(' MAE Score: %f' % (mean_absolute_error(sample_data1["label"],sample_data1["prediction"])))
print('-'*15)
print(' RMSE Score: %f' % (mean_squared_error(sample_data1["label"],sample_data1["prediction"],squared=False)))